<a href="https://colab.research.google.com/github/letddo/flower_classification/blob/main/flower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 참고 자료

- kaggle api colab으로 사용하기 - https://colab.research.google.com/github/corrieann/kaggle/blob/master/kaggle_api_in_colab.ipynb

- image to dataset - https://towardsdatascience.com/loading-custom-image-dataset-for-deep-learning-models-part-1-d64fa7aaeca6

### 1. kaggle api colab으로 사용하기
- 이제 다운도 안받아도 된다

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files 
files.upload() # kaggle 홈페이지 > 오른쪽 위 프로필 > Settings > Create new token 해서 다운받은 json 파일
# 본인 '각자' 다른 파일을 넣어줘야 합니다!

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"baksuhyun","key":"f5270d215c416a5b097995b6b23d4a86"}'}

In [ ]:
! mkdir ~/.kaggle 
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d utkarshsaxenadn/flower-classification-5-classes-roselilyetc
# kaggle competitions download -c tpu-getting-started

100% 948M/949M [00:47<00:00, 22.6MB/s]
100% 949M/949M [00:47<00:00, 21.0MB/s]


In [ ]:
!unzip flower-classification-5-classes-roselilyetc.zip

---

### 2. 데이터 다루기

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from tensorflow import keras
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential, Model
from keras.applications.resnet import ResNet50, preprocess_input
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from tensorflow.keras.optimizers.legacy import Adam

from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import random

In [ ]:
# 경로 체크 용

img = mpimg.imread('/content/Flower Classification/Flower Classification/Training Data/Daisy/Daisy (1).jpeg')
plt.imshow(img)

In [ ]:
IMG_WIDTH=224
IMG_HEIGHT=224

In [ ]:
# img -> dataset 함수
def create_dataset(img_folder):
   
    img_data_array=[]
    class_name=[]
   
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
       
            image_path= os.path.join(img_folder, dir1,  file)
            image= cv2.imread(image_path) 
            b, g, r = cv2.split(image)
            image = cv2.merge([r,g,b])
            image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
            image=np.array(image)
            image = image.astype('float32')
            image /= 255 
            img_data_array.append(image)
            class_name.append(dir1)
        
        print("Done " + dir1)
    
    
    target_dict={k: v for v, k in enumerate(np.unique(class_name))} 
    target_val=[target_dict[class_name[i]] for i in range(len(class_name))] 
    img_data_array = np.array(img_data_array) # araay -> nparray로 바꾸는 과정 
    return img_data_array, target_val

In [ ]:
x_train, t_train =create_dataset('/content/Flower Classification/Flower Classification/Training Data')
x_test, t_test =create_dataset('/content/Flower Classification/Flower Classification/Testing Data')
x_val, t_val =create_dataset('/content/Flower Classification/Flower Classification/Validation Data')

Done Rose
Done Daisy
Done Lavender
Done Lily
Done Sunflower
Done Rose
Done Daisy
Done Lavender
Done Lily
Done Sunflower
Done Rose
Done Daisy
Done Lavender
Done Lily
Done Sunflower


In [ ]:
# data test

label_names = ['Daisy', 'Lavender', 'Lily', 'Rose', 'Sunflower']
plt.figure(figsize=(10,10))

for i in range(25):
    l = random.randrange(1,5000)
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_train[l])
    plt.xlabel(label_names[t_train[l]])
plt.show()

In [ ]:
t_train = to_categorical(t_train)
t_test = to_categorical(t_test)
t_val = to_categorical(t_val)
print('One-hot Vector 적용 후 t_val shape : ', t_val.shape)
print('One-hot Vector 적용 후 t_train shape : ', t_train.shape)
print('One-hot Vector 적용 후 t_test shape : ', t_test.shape)

# 적용을 하지 않으면 아무래도 list 상태인 것 같다 

One-hot Vector 적용 후 t_val shape :  (2500, 5)
One-hot Vector 적용 후 t_train shape :  (5000, 5)
One-hot Vector 적용 후 t_test shape :  (958, 5)


## (0) 데이터 늘리기 beta

- 문제점: 메모리오류 / 수정할 것: 쪼개서 추가하기
- 사용할때 위의 코드에서 train dataset을 만드는 과정은 주석처리하고 사용해주시는게 좋습니다.

In [ ]:
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#augmentation이 적용된 image들을 시각화 해주는 함수
def show_aug_image(image, generator):
	
    # ImageDataGenerator는 여러개의 image를 입력으로 받기 때문에 4차원으로 입력 해야함.
    image_batch = np.expand_dims(image, axis=0)
    generator.fit(image_batch) 

    return image_batch


data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=(0.7, 1.3),
    horizontal_flip=True,
    vertical_flip=True,
    #rescale=1/255.0 # 학습시 적용, 시각화를 위해 임시로 주석처리
)


In [ ]:
# img -> dataset 제너레이트 하면서 
def create_dataset_with_g(img_folder):
   
    img_data_array=[]
    class_name=[]
   
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
       
            image_path= os.path.join(img_folder, dir1,  file)
            image= cv2.imread(image_path) 
            b, g, r = cv2.split(image)
            image = cv2.merge([r,g,b])
            image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
            image=np.array(image)
            image = image.astype('float32')
            image /= 255 
            img_data_array.append(image)
            class_name.append(dir1)

            for i in range(4):
              for l in show_aug_image(image, data_generator):
                img_data_array.append(l)
                class_name.append(dir1)


        print("Done " + dir1)
    
    
    target_dict={k: v for v, k in enumerate(np.unique(class_name))} 
    target_val=[target_dict[class_name[i]] for i in range(len(class_name))] 
    img_data_array = np.array(img_data_array) # araay -> nparray로 바꾸는 과정 
    return img_data_array, target_val

In [ ]:
x_train, t_train =create_dataset_with_g('./Flower Classification/Flower Classification/Training Data')

In [ ]:
t_train = to_categorical(t_train)
print('One-hot Vector 적용 후 t_train shape : ', t_train.shape)

## (0-1) 웹 크롤링

- pixabay API - https://www.youtube.com/watch?v=vzvnGT6-gPA)
- https://pixabay.com/api/docs/

- 웹 크롤링 - https://kimcoder.tistory.com/259

In [ ]:
import os
import requests
# import key

In [ ]:
key = ' ' # 님들꺼 받아쓰셈 픽사베이에서
keyword = 'daisy'
pages = 2 # 한 페이지당 20

In [ ]:
path = './crawled_img/'+ keyword + '/'

# './crawled_img/'+keyword+'_'+str(maxImages)

try:
    # 중복되는 폴더 명이 없다면 생성
    if not os.path.exists(path):
        os.makedirs(path)
    # 중복된다면 문구 출력 후 프로그램 종료
    else:
        print('이전에 같은 [검색어, 이미지 수]로 다운로드한 폴더가 존재합니다.')
        sys.exit(0)
except OSError:
    print ('os error')
    sys.exit(0)

In [ ]:
for page in range(1,int(pages)+1):
    url = 'https://pixabay.com/api/?key='+key+'&q=' +keyword+'&image_type=photo&page='+str(page)
    
    r = requests.get(url)
    json_data = r.json()
    
    for image in json_data['hits']:
        name = image['id']
        img_url = image['previewURL'] # largeImageURL(대) webformatURL(중) previewURL(소)
        r = requests.get(img_url, stream=True)
        with open(path + keyword + '(' + str(name) + ').jpg', 'wb') as f:
            f.write(r.content)

## (0-2) Image 분류기


In [ ]:
import shutil
import os
import cv2
import numpy as np
from os import remove
from  matplotlib import pyplot as plt

from tensorflow import keras
from keras.models import load_model

# 모델 불러오기
model2 = keras.models.load_model('C:\model/ResNet152v2_b64_e10.h5') # 88% 정확도
# 오류가 나면 '한글' 경로가 없는 절대경로로 입력해주세요 

In [ ]:
img_folder = './Flower Classification_/Flower Classification/Training Data/Daisy' # 수정할 이미지 폴더 # Validation Data Testing Data

img_label = 'Daisy' # 수정할 이미지의 라벨(이름) 'Daisy', 'Lavender', 'Lily', 'Rose', 'Sunflower'
move_folder = './change/' + img_label +'/' # 이동할 이미지 폴더
os.makedirs(move_folder) # 폴더가 이미 존재하면 실행 X

amount = 1.8986970e-09 # 1.8986970e-09  1.2831312e-10 # 기준값. 건드리지 않는게 좋음
IMG_WIDTH=224
IMG_HEIGHT=224 

In [ ]:
label_names = ['Daisy', 'Lavender', 'Lily', 'Rose', 'Sunflower']


worng = 0
all_img = len(os.listdir(img_folder))

print('총 파일 수: ', all_img, '\n이미지 분류 시작\n')

for file in os.listdir(img_folder):
    # all_img += 1
    image_path= os.path.join(img_folder, file)
    image= cv2.imread(image_path) 
    b, g, r = cv2.split(image)
    image = cv2.merge([r,g,b])
    image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
    image=np.array(image)
    image = image.astype('float32')
    image /= 255 
    
    output = model2.predict(image.reshape(1, IMG_WIDTH, IMG_WIDTH, 3), verbose=0) # 되려나

    if(output[0][0] + output[0][1] + output[0][2] + output[0][3] + output[0][4] - output[0][label_names.index(img_label)]  > amount or
       label_names[np.argmax(output)] != img_label):
        
        
        worng +=1
        plt.figure(figsize=(2,2))
        plt.xticks([])
        plt.yticks([])
        plt.imshow(image.reshape(IMG_WIDTH, IMG_HEIGHT, 3))
        
        print('잘못된 이미지['+ file + ']\n예측 : ' + label_names[np.argmax(output)] + ' / 정답 : ' + img_label, "\n", output)
        plt.show()
        # remove(file) # 삭제
        
        shutil.move(img_folder + '/' + file, move_folder + file)

print('잘못된 이미지 개수: ', worng, "/",  all_img)

## (1) 기존 모델

### 3. model 부분

In [ ]:
# model 부분 

width = 200
height = 200
channel = 3

model = Sequential(name = 'CIFAR10_CNN')

model.add(Conv2D(filters=100, kernel_size=(2, 2), padding= 'same', activation= 'relu',
                input_shape=(width, height, channel)))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=200, kernel_size=(2, 2), padding= 'same', activation= 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=400, kernel_size=(2, 2), padding= 'same', activation= 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate = 0.001), metrics=['accuracy'])
model.summary()

### 4. 학습하기

In [ ]:
# 학습하기
model.fit(x_train, t_train, epochs=10, batch_size=10)

### 5. 성능 확인

In [ ]:
# 성능 확인
width = 200
height = 200
channel = 3

label_names = ['Daisy', 'Lavender', 'Lily', 'Rose', 'Sunflower']

for i in range(10):
    plt.figure(figsize=(2,2))

    l = random.randrange(1,900)
    
    output = model.predict(x_test[l].reshape(1, width, height, channel))
    
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_test[l].reshape(width, height, channel))
    
    print('예측 : ' + label_names[np.argmax(output)] + ' / 정답 : ' + label_names[np.argmax(t_test[l])])
    plt.show()

In [ ]:
# 모델 평가
loss, accuracy = model.evaluate(x_test, t_test, verbose=1)
print('test loss : ', round(loss, 6))
print('test accuracy : ', round(accuracy*100, 3), '%')

## (2) 강의 실습 모델1: ResNet50

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(200,200,3))

94765736/94765736 [==============================] - 4s 0us/step


In [ ]:
x = base_model.output
x = Flatten()(x)
predictions = Dense(5, activation='softmax')(x) # Dense = 클래스

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# if one-hot -> loss = 'categorical_crossentropy'
# 1D or 정수 인코딩 대상 = sparse_categorical_crossentropy

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

#### ModelCheckpoint
- https://blog.naver.com/combioai/221468928164

In [ ]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                                           patience=5),
             keras.callbacks.ModelCheckpoint(filepath='C:\model/best_model.h5', # 적당한 폴더에 사용하길 바람
                                             monitor='val_loss',
                                             save_best_only=True)]

In [ ]:
history = model.fit(x_train, t_train, batch_size= 64, epochs=10, validation_data=(x_val, t_val), callbacks=callbacks)
#  callbacks=[early_stopping] 기존방식
#  callbacks ModelCheckpoint 방식

In [ ]:
model = keras.models.load_model('C:\model/best_model.h5') # 절대경로로 작성해야 오류가 적음

In [ ]:
# 불러오기

# 학습을 시키지 않아도 됨 
from keras.models import load_model
model2 = load_model('/content/ResNet50_1.h5')

GoogleNet

In [ ]:
from keras.applications import inception_v3
model = inception_v3.InceptionV3(weights='imagenet', include_top=True)
model.summary()

## (2) 강의 실습 모델3: VGG19Net




In [ ]:
from keras.applications.vgg19 import VGG19
tf.keras.applications.VGG19(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
model=VGG19();
model.summary();

### 5. 성능 확인

In [ ]:
# 성능 확인
width = 200
height = 200
channel = 3

label_names = ['Daisy', 'Lavender', 'Lily', 'Rose', 'Sunflower']

for i in range(10):
    plt.figure(figsize=(2,2))

    l = random.randrange(1,900)
    
    output = model.predict(x_test[l].reshape(1, width, height, channel))
    
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_test[l].reshape(width, height, channel))
    
    print('예측 : ' + label_names[np.argmax(output)] + ' / 정답 : ' + label_names[np.argmax(t_test[l])])
    plt.show()

In [ ]:
# 모델 평가
loss, accuracy = model.evaluate(x_test, t_test, verbose=1)
print('test loss : ', round(loss, 6))
print('test accuracy : ', round(accuracy*100, 3), '%')

30/30 [==============================] - 3s 85ms/step - loss: 2.8319 - accuracy: 0.4384
test loss :  2.831933
test accuracy :  43.841 %


In [ ]:
# 학습된 모델 저장 

# https://tykimos.github.io/2017/06/10/Model_Save_Load/

from keras.models import load_model

model.save('/content/ResNet501_1.h5')


### 시각화

https://stackoverflow.com/questions/41908379/keras-plot-training-validation-and-test-set-accuracy


In [ ]:
# accuracy

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# loss

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()